# Creating Fresh Data Base

In [7]:
import psycopg2

def get_create_table_queries(conn):
    create_table_queries = []
    try:
        # Create a cursor object
        cur = conn.cursor()

        # Get the list of all schemas excluding temporary and system schemas
        schema_sql = """
        SELECT schema_name 
        FROM information_schema.schemata
        WHERE schema_name NOT LIKE 'pg_temp_%'
        AND schema_name NOT LIKE 'pg_toast%'
        AND schema_name NOT LIKE 'information%'
        AND schema_name NOT LIKE 'pg_catalog%'
        """
        cur.execute(schema_sql)
        schemas = cur.fetchall()

        # Get the CREATE TABLE statements for all tables in each schema
        for schema in schemas:
            schema_name = schema[0]
            table_sql = f"""
            SELECT table_name 
            FROM information_schema.tables
            WHERE table_schema = '{schema_name}'
            """
            cur.execute(table_sql)
            tables = cur.fetchall()

            for table in tables:
                table_name = table[0]
                create_sql = f"""
                SELECT 'CREATE TABLE ' || quote_ident('{schema_name}') || '.' || quote_ident('{table_name}') || E'\n(\n' ||
                array_to_string(
                    array_agg(
                        '    ' || quote_ident(column_name) || ' ' || data_type ||
                        coalesce('(' || character_maximum_length || ')', '') ||
                        case when is_nullable = 'NO' then ' NOT NULL' else '' end
                    ), E',\n'
                ) || E'\n);\n'
                FROM information_schema.columns
                WHERE table_schema = '{schema_name}' AND table_name = '{table_name}'
                GROUP BY table_schema, table_name
                """
                cur.execute(create_sql)
                create_table_query = cur.fetchone()[0]
                create_table_queries.append(create_table_query)

        # Close the cursor
        cur.close()

    except Exception as e:
        print(f"Error: {e}")

    return create_table_queries

def save_to_sql_file(filename, queries):
    with open(filename, 'w') as f:
        for query in queries:
            f.write(query + '\n\n')


conn = psycopg2.connect(
    host="192.168.2.54",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# create the schemas


create_table_queries = get_create_table_queries(conn)
save_to_sql_file("create_tables.sql", create_table_queries)


# Close the connection
conn.close()

# Run this cell to create all the tables and schemas

In [2]:
import psycopg2
# connect to local host db named BravisaDB
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# run the create_table.sql file 
with open("create_tables.sql", 'r') as f:
    create_table_sql = f.read()
    cur = conn.cursor()
    cur.execute(create_table_sql)
    conn.commit()
    cur.close()
    